In [1]:
import sys
!{sys.executable} -m pip install torch==1.10.1
!{sys.executable} -m pip install torchvision==0.11.2
!{sys.executable} -m pip install torchaudio==0.10.1
!{sys.executable} -m pip install cython
!{sys.executable} -m pip install packaging
!{sys.executable} -m pip install mxnet==1.7.0.post2
!{sys.executable} -m pip install gluonnlp
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install tqdm
!{sys.executable} -m pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!{sys.executable} -m pip install openpyxl
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install ipywidgets
!{sys.executable} -m pip install re soynlp emoji



  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /private/var/folders/gy/xty89whd5vz58g10775h1l_w0000gn/T/pip-req-build-d0eldf83
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /private/var/folders/gy/xty89whd5vz58g10775h1l_w0000gn/T/pip-req-build-d0eldf83
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [2]:
import torch 
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.auto import tqdm
import pandas as pd
import sklearn
import re

In [3]:
#KoBERT
from kobert.pytorch_kobert import get_pytorch_kobert_model
#KoBERT-tokenizer불러오기
from kobert.utils import get_tokenizer
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
#GPU 사용
#device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /Users/gangjuhyeon/Dev/SKKU/딥러닝비정형/.cache/kobert_v1.zip
using cached model. /Users/gangjuhyeon/Dev/SKKU/딥러닝비정형/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [4]:
#csv_comments_data = pd.read_csv('/Users/gangjuhyeon/Dev/SKKU/딥러닝비정형/masked_comments_total_data.csv')
# creating an output excel file
#csv_comments_data.to_excel('masked_comments_total_data.xlsx', index=False)

comments_data =  pd.read_excel('/Users/gangjuhyeon/Dev/SKKU/딥러닝비정형/masked_comments_total_data.xlsx')
headlines_data = pd.read_excel('/Users/gangjuhyeon/Dev/SKKU/딥러닝비정형/labelledNewsData.xlsx')

In [5]:
comments_data.sample(n=5)

,Unnamed: 0,Date,Sentence,Sentiment_Num,Num
3595,3595,23.05.05,담가승부는 없어,0.5,262.0
3408,3408,23.05.04,청소기로 빨아도,0.5,75.0
1573,1573,23.04.27,멧돼지의 주먹구구식 계산법,정치,130.0
310,310,23.05.05,이것도,0.5,NaN
3467,3467,23.05.04,외국인 물량을 개인이 빼앗아 오는건 국가적으로 아주 바람직한 일 핵인싸 공감글,0.75,134.0


In [6]:
headlines_data.sample(n=5)

,Unnamed: 0,Num,Date,Sentence,Sentiment_Num
3044,3044,134,2023-05-01 07:01:01,고병철 포스텍 홀딩스 투자대표 그의 무기는 솔직함과 명확함이다 C감소스위트,0.75
3057,3057,147,2023-04-16 17:04:01,2차전지에 뜨거워진 포스코 올해 시총 22조원 카카오도 제쳤다,1.00
151,151,152,2023-05-04 16:59:00,울산에너지고 학생들 삼성전자 마이스터 장학생 합격,0.50
2647,2647,37,2023-05-06 10:35:35,삼성전자 이마트에서 청소기 구매 시 스쿨존 안심 가방커버 및 한정판 과자 패키지 증정,0.50
2866,2866,256,2023-05-02 11:47:21,삼성전자감소크라운제과 어린이 교통사고 예방 캠페인 실시,0.50


In [7]:
# droping unnecessary columns
commentsData = comments_data.drop(['Unnamed: 0'],axis = 1)
commentsData = commentsData.drop(['Num'],axis = 1)
headlinesData = headlines_data.drop(['Unnamed: 0'],axis = 1)
headlinesData = headlinesData.drop(['Num'],axis = 1)

In [8]:
commentsData.sample(n=5)

,Date,Sentence,Sentiment_Num
1420,23.05.01,NAVER 증권 리서치 종목분석 리포트 발행 현황 2023 4,0.5
524,23.05.05,Msci 편입 확실 한것 같습니다,0.75
2447,23.04.27,2차전지는 신고가 치고 훨훨 날라가네,1
1982,23.04.27,삼성,0.5
2991,23.05.01,예전에 주식고수대회,0.5


In [9]:
headlinesData.sample(n=5)

,Date,Sentence,Sentiment_Num
1428,2023-04-26 17:32:02,SK하이닉스 3조4천억 적자 2분기가 바닥,0.00
1650,2023-04-26 09:47:05,2Q 메모리가격 급등 어려워 감산계획 유지 감소SK하이닉스 컨콜,0.25
3453,2023-04-26 19:59:57,단독 현대차 러시아 전면 철수 공장 매각 마무리 단계,0.00
1487,2023-04-26 09:45:01,SK하이닉스 어닝 쇼크 1분기 실적 역대 최악,0.00
3234,2023-05-04 11:07:02,컨콜 SK이노 현대차와의 JV 투자금 이미 반영됐다,0.50


In [10]:
commentsData.loc[(commentsData['Sentiment_Num'] == 0.0), 'Sentiment_Num'] = 0  #매우 부정 => 0
commentsData.loc[(commentsData['Sentiment_Num'] == 0.25), 'Sentiment_Num'] = 1  #다소 부정 => 1
commentsData.loc[(commentsData['Sentiment_Num'] == 0.5), 'Sentiment_Num'] = 2  #중도적 => 2
commentsData.loc[(commentsData['Sentiment_Num'] == 0.75), 'Sentiment_Num'] = 3  #다소 긍정 => 3
commentsData.loc[(commentsData['Sentiment_Num'] == 1.0), 'Sentiment_Num'] = 4  #매우 긍정 => 4
commentsData.loc[(commentsData['Sentiment_Num'] == '정치'), 'Sentiment_Num'] = 5  #무관/정치 => 5


headlinesData.loc[(headlinesData['Sentiment_Num'] == 0.00), 'Sentiment_Num'] = 0  #매우 부정 => 0
headlinesData.loc[(headlinesData['Sentiment_Num'] == 0.25), 'Sentiment_Num'] = 1  #다소 부정 => 1
headlinesData.loc[(headlinesData['Sentiment_Num'] == 0.50), 'Sentiment_Num'] = 2  #중도적 => 2
headlinesData.loc[(headlinesData['Sentiment_Num'] == 0.75), 'Sentiment_Num'] = 3  #다소 긍정 => 3
headlinesData.loc[(headlinesData['Sentiment_Num'] == 1.00), 'Sentiment_Num'] = 4  #매우 긍정 => 4

In [11]:
commentsData.sample(n=5)

,Date,Sentence,Sentiment_Num
2358,23.04.26,기약없는 4만전자 5만전자 삼성전자 개인들은 자선사업가들인가,2
1107,23.05.05,윤대통령님 잘하고 있어요,5
1339,23.04.28,시총 40조가 적정 주가임,4
2025,23.04.20,기관 특히,2
533,23.05.05,위메이드 차트 보고 와라,2


In [12]:
headlinesData.sample(n=5)

,Date,Sentence,Sentiment_Num
1238,2023-04-17 14:41:01,새만금에 2차전지 큰 거 온다 LG화학 SK온 등 투자 몰려,3.0
3067,2023-04-17 11:24:49,광양시의회 거리에서 포스코는 상생하라 촉구,4.0
1596,2023-04-20 11:13:04,SK하이닉스 D램 칩 12개 쌓았다 AI 챗봇 수요 맞춰 하반기 공급,4.0
2973,2023-04-27 16:19:07,포스코홀딩스 1분기 영업익 7047억,2.0
1570,2023-04-26 10:04:03,컨콜 SK하이닉스 D램 한 자릿수 중후반 낸드 10 중후반 성장 예상,3.0


def remove_words_from_sentences(sentences, words_to_remove):
    cleaned_sentences = []
    for sentence in sentences:
        words = sentence.split()
        cleaned_words = [word for word in words if word not in words_to_remove]
        cleaned_sentence = ' '.join(cleaned_words)
        cleaned_sentences.append(cleaned_sentence)
    return cleaned_sentences


In [13]:
comments_data_list = []
for q, label in zip(commentsData['Sentence'], commentsData['Sentiment_Num'])  :
    data = []
    data.append(str(q)) # KoBERT TypeError: Argument 'text' has incorrect type (expected str, got float)
    data.append(int(label)) # KoBERT ValueError: invalid literal for int() with base 10: '4.0'

    comments_data_list.append(data)

print(comments_data_list)

[['미국 I R A 최대수혜주식', 4], ['제이의 반도체 이차전지', 2], ['IRA발표로 에코프로 앨앤에프sk온은 재료소멸 ', 0], ['외인', 2], ['전고체 밧데리 ', 4], ['생산세액공제 AMPC  예상 수혜 규모', 3], ['스텔란티스   삼성SDI 는 북미지역 합작공장 투자', 4], ['삼성전고체 배터리', 4], ['Ira로 자세히 아시는분 그리고 월요일 주가', 2], ['중남미 리튬 확보 기업이', 2], ['추정 PER 단순비교', 2], ['삼성SDI  중국 연구소 설립 초격차 기술경쟁력 강화', 4], ['장판막에 외인들이 들어와서', 4], ['오늘공매도', 0], ['장막판되니', 2], ['배터리 장비업체들이 지금 콧노래 부르는 5가지 이유', 4], ['광우병  오염수를 통해 본 좌뻘 놈들의 특징 ', 5], ['막산이 같은 것이 4 3 기념식에 참가하는 것은', 0], ['오늘의 매도주체는', 0], ['오늘 익절 하신 고객님 ', 0], ['이제', 2], ['아메바 단타개미들', 0], ['삼성SDI  중국 연구소 설립 LFP 개발 신호탄 ', 4], ['한국경제는 수출이 전부인데     전년대비감소23  금일 코스피가 힘이 없습니다 ', 0], ['2차전지 관련주  잔칫 날  미 IRA 세부지침에 동반 급등', 4], ['애널들 또 매수 리포트 쓰기 시작', 4], ['80마넌', 4], ['30평분양가 9억대 원리금매월300씩내고 살아바라', 0], ['삼성SDI 리포트   우리만 모를 뿐  캐파 상향 계획은 진행 중', 4], ['낼부터 니가 대장 잡는다며', 0], ['불안하게', 0], ['오늘 이상하게 팔고싶더라니', 0], ['KB증권  이데일리 가사 비판    ', 0], ['종가 망치캔들', 3], ['신한 씨에스윈드   EU 정책 수혜 극대화 위해 증설 진행 목표가 8만7000원', 4], ['환율 참조  국제 유가 참조 닥터쿠퍼 구리값 참조', 0], ['재료소멸 오늘 종가는 망치캔들',

In [14]:
headlines_data_list = []
for q, label in zip(headlinesData['Sentence'], headlinesData['Sentiment_Num'])  :
    data = []
    data.append(str(q))
    data.append(int(label)) # # KoBERT ValueError: invalid literal for int() with base 10: '4.0'

    headlines_data_list.append(data)
    
print(headlines_data_list)

[['LG엔솔  자신감   삼성SDI  확장   SK온  반전  배터리3사 노림수', 4], ['SK온  2분기엔 함께 웃나 LG엔솔 삼성SDI  1분기 활짝', 4], ['삼성SDI  1분기 역대 최대 실적 3개 분기 연속 매출 5조 돌파', 4], ['삼성SDI  GM과 4조원 배터리 합작법인 설립', 3], ['배터리 업고 승승장구 LG엔솔 삼성SDI  1Q 매출  역대 최대', 4], ['삼성SDI  GM과 배터리 합작법인 설립', 3], ['R amp D만이 살 길 삼성SDI 카카오  1조 풀었다', 4], ['삼성SDI 1분기 기준 역대 최대실적 3분기 연속 5조 매출 종합2보 ', 4], ['배터리  삼성SDI 1분기 역대최대 실적  북미 사업 본격화  종합 ', 4], ['에코프로 삼성SDI  국내 최대규모 LFP 프로젝트 맡았다', 3], ['삼성SDI  GM과 배터리 합작법인 설립  30억 달러 투자 ', 3], ['삼성SDI는 에 4조  LG엔솔은 한국에 6000억 계속되는 K배터리 투자', 3], ['삼성SDI  1분기 매출 5조3548억원 3분기 연속 5조 돌파', 4], ['에너지 밀도 10  높인 P6  2024년 양산 목표 감소삼성SDI 컨콜', 3], ['삼성SDI  GM과 미국에 배터리 공장 세운다', 2], ['삼성SDI  니켈 90  이상 차세대 배터리  2024년 양산 ', 4], ['삼성SDI   진출 속도 내고 제품 포트폴리오도 늘린다  종합 ', 4], ['삼성SDI 우수인재 확보 나서 경력사원 대규모 채용', 4], [' 컨콜  삼성SDI  LFP 배터리 등 포트폴리오 확대 ', 3], ['삼성SDI  GM과의 북미 JV 설립 2분기 완료 ', 3], [' K배터리 하드캐리  초격차 신봉자  삼성SDI  소부장박대리 ', 2], ['LFP 배터리 등 제품 포트폴리오 확대 계획 감소삼성SDI 컨콜', 4], ['삼성 SK하닉  반도체 리스크 완화  금주의 산업계 이슈 ', 3], ['키움 삼성SDI  우상향 이익 지속

In [15]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
commentsDataset_train, commentsDataset_test = train_test_split(comments_data_list, test_size=0.25, random_state=0)
headlinesDataset_train, headlinesDataset_test = train_test_split(headlines_data_list, test_size=0.25, random_state=0)

In [16]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5 # 분류는 이거 중요하다던데 

In [17]:
#댓글토큰화
#from transformers import AutoTokenizer, AutoModel
#commentsTokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
#제목토큰화
from kobert.utils import get_tokenizer
titleTokenizer = get_tokenizer()
commentTokenizer = get_tokenizer()
commentsTok = nlp.data.BERTSPTokenizer(titleTokenizer, vocab, lower=False)
titleTok = nlp.data.BERTSPTokenizer(commentTokenizer, vocab, lower=False)


using cached model. /Users/gangjuhyeon/Dev/SKKU/딥러닝비정형/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /Users/gangjuhyeon/Dev/SKKU/딥러닝비정형/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


class BERTDataset:
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        import gluonnlp as nlp
        import numpy as np
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
from bertdataset import BERTDataset

In [19]:
commentsDataset_trained = BERTDataset(commentsDataset_train, 0, 1, commentsTok, max_len, True, False)
commentsDataset_tested = BERTDataset(commentsDataset_test, 0, 1, commentsTok, max_len, True, False)
headlinesDataset_trained = BERTDataset(headlinesDataset_train, 0, 1, titleTok, max_len, True, False)
headlinesDataset_tested = BERTDataset(headlinesDataset_test, 0, 1, titleTok, max_len, True, False)

In [20]:
# 토큰화와 패딩이 잘 이루어졌는지 확인
commentsDataset_trained[0]
commentsDataset_tested[0]


(array([   2, 2653, 5341, 6607, 7565, 6003, 7010, 7234, 5468, 3234, 1113,
        3401, 4648, 3043,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [ ]:
headlinesDataset_trained[0]
headlinesDataset_tested[0]

In [21]:
comments_trained_dataloader = torch.utils.data.DataLoader(commentsDataset_trained, batch_size=batch_size, num_workers=5)
comments_tested_dataloader = torch.utils.data.DataLoader(commentsDataset_tested, batch_size=batch_size, num_workers=5)
headlines_trained_dataloader = torch.utils.data.DataLoader(headlinesDataset_trained, batch_size=batch_size, num_workers=5)
headlines_tested_dataloader = torch.utils.data.DataLoader(headlinesDataset_tested, batch_size=batch_size, num_workers=5)


import torch
import torch.nn as nn

class BERTCommentsClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정, 댓글데이터 분류는 정치댓글을 포함한 6중분류##
                 dr_rate=None,
                 params=None):
        super(BERTCommentsClassifier, self).__init__()
        import torch 
        import torch.nn as nn
        import torch.nn.functional as F
        import torch.optim as optim
        from torch.utils.data import Dataset, DataLoader
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        import torch 
        import torch.nn as nn
        import torch.nn.functional as F
        import torch.optim as optim
        from torch.utils.data import Dataset, DataLoader
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
    


class BERTTitlesClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,   ##클래스 수 조정, 뉴스기사제목은 5중분류##
                 dr_rate=None,
                 params=None):
        super(BERTTitlesClassifier, self).__init__()
        import torch 
        import torch.nn as nn
        import torch.nn.functional as F
        import torch.optim as optim
        from torch.utils.data import Dataset, DataLoader
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        import torch 
        import torch.nn as nn
        import torch.nn.functional as F
        import torch.optim as optim
        from torch.utils.data import Dataset, DataLoader
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [22]:
from bertclassifier import BERTCommentsClassifier, BERTTitlesClassifier

In [23]:
#제목분석에 사용할 BERT 모델 불러오기
titleModel = BERTTitlesClassifier(bertmodel, dr_rate=0.5)#.to(device) GPU쓰실거면 ㄱㄱ
commentsModel = BERTCommentsClassifier(bertmodel, dr_rate=0.5)
#commentsModel = AutoModel.from_pretrained("beomi/KcELECTRA-base")

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']



comments_optimizer_grouped_parameters = [
    {'params': [p for n, p in commentsModel.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in commentsModel.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
]
comments_optimizer = AdamW(comments_optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

cmts_t_total = len(comments_trained_dataloader) * num_epochs
cmts_warmup_step = int(cmts_t_total * warmup_ratio)
cmts_scheduler = get_cosine_schedule_with_warmup(comments_optimizer, num_warmup_steps=cmts_warmup_step, num_training_steps=cmts_t_total)



titles_optimizer_grouped_parameters = [
    {'params': [p for n, p in titleModel.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in titleModel.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
]
titles_optimizer = AdamW(titles_optimizer_grouped_parameters, lr=learning_rate)

hdls_t_total = len(headlines_trained_dataloader) * num_epochs
hdls_warmup_step = int(hdls_t_total * warmup_ratio)
hdls_scheduler = get_cosine_schedule_with_warmup(titles_optimizer, num_warmup_steps=hdls_warmup_step, num_training_steps=hdls_t_total)


#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    import torch 
    import torch.utils.data
    import numpy
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [24]:
from calc_accuracy import calc_accuracy

In [ ]:
# 코멘트 학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    commentsModel.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(comments_trained_dataloader)):
        comments_optimizer.zero_grad()
        token_ids = token_ids.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        segment_ids = segment_ids.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        valid_length= valid_length
        label = label.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        out = commentsModel(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(commentsModel.parameters(), max_grad_norm)
        comments_optimizer.step()
        cmts_scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(comments_tested_dataloader)):
        token_ids = token_ids.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        segment_ids = segment_ids.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        valid_length= valid_length
        label = label.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        out = commentsModel(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
    

  0%|          | 0/43 [00:08<?, ?it/s]

epoch 1 batch id 1 loss 1.8847925662994385 train acc 0.171875


자꾸 AttributeError: Can't get attribute 'f' on <module '__main__' (built-in)> 식의 오류가 떠서 대체 무슨 일이 일어나는지 알아봤다.
ipython의 multiprocessing 문제라고 한다. 
https://jaksam.tistory.com/30
loss값이 10^-5면 엄청 조은거


In [ ]:
#headlines 학습
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    titleModel.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(headlines_trained_dataloader)):
        titles_optimizer.zero_grad()
        token_ids = token_ids.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        segment_ids = segment_ids.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        valid_length= valid_length
        label = label.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        out = titleModel(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(titleModel.parameters(), max_grad_norm)
        titles_optimizer.step()
        hdls_scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    titleModel.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(headlines_tested_dataloader)):
        token_ids = token_ids.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        segment_ids = segment_ids.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        valid_length= valid_length
        label = label.long()#.to(device) #GPU쓰실거면 .to(device)ㄱㄱ
        out = titleModel(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

7. 새로운 문장 테스트

import torch 
import torch.utils.data
import torch.nn as nn
import numpy
import gluonnlp as nlp
from kobert.utils import get_tokenizer
from bertdataset import BERTDataset

#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long()#.to(device)
        segment_ids = segment_ids.long()#.to(device)

        valid_length= valid_length
        label = label.long()#.to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("매우 부정적")
            elif np.argmax(logits) == 1:
                test_eval.append("다소 부정적")
            elif np.argmax(logits) == 2:
                test_eval.append("중도적")
            elif np.argmax(logits) == 3:
                test_eval.append("다소 긍정적")
            elif np.argmax(logits) == 4:
                test_eval.append("매우 긍정적")
            elif np.argmax(logits) == 5:
                test_eval.append("긍부정과 무관하거나 정치적")


        print(">> 입력하신 내용에서 " + test_eval[0] + "인 어조가 느껴집니다.")

In [ ]:
from predictor import predict